In [1]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sys.path.append('/envs')
sys.path.append('/models')
import envs.viper as vpr
import models.models as models
import models.dqn as dqn

In [ ]:
import importlib
importlib.reload(vpr); importlib.reload(models); importlib.reload(dqn)

In [ ]:
viper_env = vpr.Viper(_sks = 798)

In [5]:
torch.cuda.empty_cache()

In [6]:
#state = viper_env.state()
#state[None, :]

In [7]:
#state.unsqueeze(0)

In [ ]:
agent = dqn.DQN(viper_env, _max_history=2048)

In [ ]:
viper_env.compute_gcd(2.5, 798, 15)

In [10]:
action_sequence=[1, 3, 6, 9, 0, 2, 5, 0, 0]

In [ ]:
print(viper_env.state())
for action in action_sequence:
    print(viper_env.step(action, _verbose=True))
    print(viper_env.state())

In [ ]:
agent.env.reset_env()
print(agent.env.state())
print(agent.env.valid_actions())
print([np.random.choice(agent.env.valid_actions()[1])])
print(agent.env.gcd)

In [ ]:
agent.get_action(agent.env.state(), e=1)[0]

In [ ]:
history_x, history_y = agent.train(gamma=0.50, num_epochs=50, session_limit=100)

In [ ]:
fig, ax = plt.subplots()
fig.suptitle('Rewards')
ax.plot(history_x, history_y)
ax.legend(['Train Rewards', 'Eval Rewards'])
ax.set_xlabel('Epoch')
ax.set_ylabel('Rewards')
plt.savefig('rewards_graph.png', bbox_inches='tight')
plt.show()

In [ ]:
agent.load_checkpoint(f'./checkpoints/_{agent.model_name}_best.pth')

In [ ]:
agent.test(num_steps=50)

In [ ]:
agent.predict(agent.env.state())

In [ ]:
agent.env.state()